<a name="top"></a>

# Download and tidy data

## Contents:
* [Import Packages + Set up Paths](#import)


* [Connect to Mongo](#mongo)  


* [Save Dataframes from Each Experiment](#initialize)  
    * [Experiment 1.0](#exp1_data): exit survey, solve (test), compare (pre/posttest)


* [Read Dataframes Back Up](#read_from_csv)
    * Can start here to read from locally stored data and save time


* [Summary Statistics](#summary_stats)
    * [Summary of the Data](#summary): Also includes some simple exploratory statistical tests


# <a name="import"></a> Import Packages + Settings([^](#top))

In [ ]:
# packages
import os, sys
import pymongo as pm
import numpy as np
import scipy.stats as stats
import math
import pandas as pd
import json
import socket

from operator import itemgetter
from collections import Counter

import seaborn as sns

from IPython.display import clear_output

## add helpers to python path
proj_dir =  os.path.abspath('../..')
sys.path.append(os.path.join(proj_dir))
from utils import sokoban_solvers as ss
from utils import analysis_utils as au

In [ ]:
# notebook parameters
sns.set_context('talk')
sns.set_style('white')

project_name = 'fun-puzzles'
experiment_name = 'fun-puzzles-exp1'
iterationName = 'production2'
load_from_mongo = False # if resync data
only_completed = True # only keep complete sessions with exit survey

do_replacements = False # update used stim to numGames=100?

np.random.seed(123)


# directory and file hierarchy
studyName = 'study2'
# data directory
data_dir = os.path.join(proj_dir, 'data', studyName)
def make_dir_if_not_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name
make_dir_if_not_exists(data_dir)

# <a name="mongo"></a> Connect To Mongo ([^](#top))

In terminal, connect to data server by running

`ssh -fNL 27017:127.0.0.1:27017 junyichu@cogtoolslab.org`

In [ ]:
# set vars
auth = pd.read_json('../../../auth.json', typ='series') # this auth.json file contains the password
# auth = pd.read_json(os.path.join(proj_dir,'auth.json'), typ='series') # this auth.json file contains the password
pswd = auth.password
user = auth.user
host = 'cogtoolslab.org'

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1:27017')
db = conn[project_name]

# <a name="initialize"></a> Save Dataframes from Each Experiment ([^](#top))

### Comprehension Checks

In [ ]:
# Comprehension Checks
if load_from_mongo:    
    coll = db[experiment_name]
    print("Fetching comprehension trials...")
    clear_output(wait=True)
    compre_cursor = coll.find({'study_metadata.project': project_name,
                'study_metadata.experiment': experiment_name,
                'study_metadata.iteration':  iterationName,#{'$in': iterationNames},
                'study_phase': 'comprehension'})
    compre = pd.DataFrame(list(compre_cursor))

    compre_tidy = (pd.DataFrame(compre.session_info.tolist())
                   .join(pd.DataFrame(compre.response.tolist(), index=compre.index))
                   .join(pd.DataFrame(compre[['rt', 'time_elapsed']], index=compre.index))
                   )
    compre_tidy['comprehension_attempt'] = compre['trial_index'].apply(lambda x: (x/2) - 3) # 8, 10 -> 1, 2
    compre_tidy['q0correct'] = compre['trial_index'].apply(lambda x: (x/2) - 3) # 8, 10 -> 1, 2
    # code accuracy
    compre_tidy['Q0correct'] = compre_tidy.apply(lambda x: 1 if x['condition'] in x['Q0'] else 0, axis=1)
    compre_tidy['Q1correct'] = compre_tidy.apply(lambda x: 1 if 'solve' in x['Q1'] else 0, axis=1)
    compre_tidy['correct'] = compre_tidy['Q1correct'] * compre_tidy['Q0correct']
    
    print("Correct on first attempt: {}".format(compre_tidy['correct'][compre_tidy['comprehension_attempt']==1].sum()))
    print("Correct on second attempt: {}".format(compre_tidy['correct'][compre_tidy['comprehension_attempt']==2].sum()))
    print("Incorrect on second attempt: {}".format(np.sum(compre_tidy['correct'][compre_tidy['comprehension_attempt']==2] == 0)))

    compre_tidy.to_csv(os.path.join(data_dir,iterationName+'_comprehension.csv'),index=False)


### Get exit survey

Only participants who completed the entire study will have an exit survey json.

In [ ]:
if load_from_mongo:
    # Get exit survey and study timing data
    print("Fetching exit survey...")
    clear_output(wait=True)
    e = coll.find({'study_metadata.project': project_name,
                'study_metadata.experiment': experiment_name,
                'study_metadata.iteration': iterationName,#{'$in': iterationNames},
                'study_phase': 'exit survey',
                'study_metadata.dev_mode': False})
    E = pd.DataFrame(list(e))

    # get list of valid game IDs (i.e, subject number)
    completed_gameIDs = [val['gameID'] for key, val in E['session_info'].items() if 'gameID' in val]

    print("There are {} completed games.".format(len(completed_gameIDs)))

    # save as tidy csv
    E_tidy = (pd.DataFrame(E.study_metadata.tolist(), index=E.index)
            .drop(['dev_mode', 'study_duration', 'compare_stim_duration'], axis=1)
            .join(pd.DataFrame(E.session_info.tolist(), index=E.index))
            .join(pd.DataFrame(E.session_timing.tolist(), index=E.index))
            .join(pd.DataFrame(E.response.tolist(), index=E.index)))
    
    E_tidy.to_csv(os.path.join(data_dir,iterationName+'_survey.csv'),index=False) # anonymize

### Get main puzzle-solving data

We'll make two dataframes: 

- Trial info: one csv that has one row per trial
- Trace data: a folder of csvs, one csv per gameID, one row per action (move / undo / restart / cannot step, etc.)

In [ ]:
# Trial info dataframe

if load_from_mongo:    
# ## fetch main solving records that match desire iterationNames, and turn it into a dataframe
    print("Fetching test (select-solve-rate) phase...")
    clear_output(wait=True)
    s = coll.find({'study_metadata.project': project_name,
                'study_metadata.experiment': experiment_name,
                'study_metadata.iteration':  iterationName,#{'$in': iterationNames},
                'study_phase': 'test'})
    S = pd.DataFrame(list(s))

    # parse trial number
    S = S.assign(trialNum = S['progress_prompt'].str.split(' ').str[1])
    S.trialNum = S.trialNum.astype(int)
    # S = S.assign(gameID = appS['session_info']['gameID'])
    S['gameID'] = S.apply(lambda row: row['session_info'].get('gameID'), axis=1)

    # Begin building a tidy dataframe
    S_tidy = (pd.DataFrame(S.study_metadata.tolist())
              .drop(['dev_mode', 'study_duration', 'compare_stim_duration'], axis=1)
              .join(pd.DataFrame(S.session_info.tolist(), index=S.index))
              .join(S[['study_phase', 'trialNum', 'trial_type', 'response', 'rt', 'solveDuration','steps', 'inputEvents']])
              .join(pd.json_normalize(S['stimuli'])))
    S_gameIDs = set([val['gameID'] for key, val in S['session_info'].items() if 'gameID' in val])
    print("Fetched {} jspsych frames from {} games".format(len(S_tidy), len(S_gameIDs)))

    # make one row per trial, not 3  
    S_solve= S_tidy.loc[S_tidy['trial_type']=='sokoban-solve',['gameID', 'trialNum', 'solveDuration', 'steps', 'inputEvents']]
    # Extract basic metrics of solve performance
    S_solve = S_solve.assign(solved = np.where(S_solve['solveDuration'].notna(), 1, 0))
    S_solve = S_solve.assign(attempt_nsteps = S_solve['steps'].apply(lambda x: len(x) if isinstance(x, list) else pd.NA))
    S_solve['attempt_nInputEvents'] = S_solve['inputEvents'].apply(lambda x: len(x) if isinstance(x, list) else pd.NA)
    S_solve['boxesSolved'] = S_solve['inputEvents'].apply(lambda x: au.maxBoxOnGoal(x) if isinstance(x, list) else pd.NA)

    
    S_solve = S_solve.drop(['steps', 'inputEvents'], axis=1)

    S_select = (S_tidy.loc[S_tidy['trial_type']=='sokoban-select',['gameID', 'trialNum', 'response', 'rt']]
                .rename(columns={"response": "select_response", "rt": "select_rt"}))

    S_rate = (S_tidy.loc[S_tidy['trial_type']=='sokoban-rate',['gameID', 'trialNum', 'response', 'rt']]
                .rename(columns={"response": "rate_response", "rt": "rate_rt"}))
    
    S_tidy2 = (S_tidy[S_tidy['trial_type']=='sokoban-rate']
               .drop(['trial_type', 'response', 'rt', 'solveDuration', 'steps', 'inputEvents'], axis=1)
               .merge(S_select)
               .merge(S_rate)
               .merge(S_solve))
    

    ## only keep complete sessions
    if (only_completed):
        if len(S_gameIDs) > len(completed_gameIDs):
            S_tidy2 = S_tidy2[S_tidy2['gameID'].isin(completed_gameIDs)]
        print("Keeping {} trials from {} completed games".format(len(S_tidy2), len(completed_gameIDs)))
    else:
        print("Keeping {} trials from {} games".format(len(S_tidy2), len(S_gameIDs)))
    
    # export to csv
    S_tidy2['level_name'] = S_tidy2['level_name'].astype(int, errors='ignore').astype(str)

    S_tidy2.to_csv(os.path.join(data_dir,iterationName+'_testTrials.csv'),index=False) # anonymize

Now export trace data as one csv per puzzle, organized by gameID. 

First check: how many trials of trace data do we have per participant?

In [ ]:
print("N Trials per participant with trace data: {}".format(S_tidy2.value_counts('gameID').value_counts()))

print("Who doesn't have 8 trials?")
S_tidy2.value_counts('gameID')[S_tidy2.value_counts('gameID') < 8]

In [ ]:
def generate_boards(trace_df, method, board0):
    """
    Create a DataFrame of box traces based on the specified method.
    Args:
        trace_df (pd.DataFrame): The input DataFrame containing traces.
        method (str): The method to create box traces ('from_df' or 'computed').
        boxes (optional): Optional boxes data if method is 'computed'.
    Returns:
        pd.DataFrame: A DataFrame of box traces.
    """
    if method == "from_df":
        # unlist the boxes variable, so now one row per box
        boxes = trace_df.explode("boxes")["boxes"]
        boxes2 = (
            pd.json_normalize(boxes)
            .set_index(boxes.index)
            .reset_index(names="eventNum")
        )  # get original action index as a separate column
        nboxes = len(boxes[0])
        boxes2['boxID'] = np.tile(range(1, nboxes + 1), int(len(boxes2) / nboxes)).astype('str') # index refers to event number [0,0,0,1,1,1]; add box number
        # pivot wider, so back to one event per row
        boxes3 = boxes2.pivot(index='eventNum', columns = 'boxID', values = ['x', 'y', 'onGoal'] )
        boxes3.columns = boxes3.columns.to_flat_index().str.join('_') # rename columns to x_1, x_2, etc.
        print(boxes3.head(5))
        return boxes3.reset_index()
    elif method == "computed":
        if board0 is None:
            raise ValueError("Level layout not provided")
        
        board_stack = [board0]
        boards = []
            
        for i, row in trace_df.iterrows():
            if row['action'] == "restart":
                board_stack=[board0] # clear boards stack
            elif row['event'] in ["cannot_step" , "cannot_push"]:
                None # nothing happens to board stack
            elif row['action'] == "undo":
                if len(board_stack) > 1:
                    board_stack.pop() # remove last board until only initial board remains
            else: # movement happened
                new = board_stack[-1].clone()
                new.update_board_from_arrow(row['action'].lower())
                board_stack.append(new)
                
            # now update boxes
            currentBoard = board_stack[-1].clone()
            boards.append(currentBoard)

        return boards
    else:  
        raise ValueError("Invalid method specified")


# # testing
# game = '2485-47f57e30-a80b-4dbd-a9e6-6b306a349c3f'
# trialnum = 7
# row = S[(S['gameID'] == game) & (S['trialNum'] == trialnum) & (S['trial_type'] == 'sokoban-solve')]
# traces = pd.json_normalize(row['inputEvents'].explode(ignore_index=True)).reset_index(names="eventNum")
# # update time column
# start_time = row['startTime'].item()
# traces['secondsElapsed'] = .001 * (traces['timestamp'] - start_time)
# traces.drop(['timestamp'], axis=1, inplace=True)
# # get initial board info
# board0 = ss.State().stringInitialize(row['stimuli'].values[0]['layout']) # create board object
# value0 = board0.getHeuristic()
# # update all actions 
# boards = generate_boards(traces, "computed", board0 = board0)
# traces['boxes'] = [b.crates for b in boards]
# traces['state_key'] = [b.getKey2() for b in boards]
# traces['state_value'] = [b.getHeuristic() for b in boards]
# traces['restartCount'] = (traces['action'] == 'restart').cumsum()
# # traces['value_diff'] = traces['state_value'].diff()
# # traces.loc[traces.index[0], 'value_diff'] = traces.loc[traces.index[0], 'state_value'] - value0
# # add first row for initial board
# initial_row = pd.DataFrame([{'eventNum': None, 'action': None, 'event': None, 'boxes': board0.crates, 'agent.x': board0.player['x'], 'agent.y': board0.player['y'], 'agent.orientation': 'U', 'secondsElapsed': 0, 'state_key': board0.getKey2(), 'state_value': value0, 'restartCount': 0}])
# df = pd.concat([initial_row, traces], ignore_index=True).reset_index(names='stateNum')
# df.head()
# # traces.head()

In [ ]:
if load_from_mongo:    
    print("Parsing puzzle-solving traces...")
    clear_output(wait=True)

    # For each game, create a folder and save one csv per puzzle
    # in the format: gameID-xxx_trial-xxx_puzzle-xxx_inputevents.csv
    # where each row of the csv has input, game state
    # for game in ['0043-c146f09f-8bb2-4ccc-9577-9363bf3422c1']:
    for game in completed_gameIDs:
        # make dir for each subject
        game_dir = os.path.join(data_dir, "traces", game)
        make_dir_if_not_exists(game_dir)
        
        for trialnum in range(1, 9): # 1 to 8
            row = S[(S['gameID'] == game) & (S['trialNum'] == trialnum) & (S['trial_type'] == 'sokoban-solve')]
            if not row.empty:
                clear_output(wait=True)
                # build out traces object
                start_time = row['startTime'].item()
                traces = pd.json_normalize(row['inputEvents'].explode(ignore_index=True)).reset_index(names="eventNum")
                traces['gameID'] = game
                traces['trialNum'] = trialnum
                if 'timestamp' in traces.columns:
                    print("parsing {} trial {}".format(game, trialnum)) 
                    # update time column
                    traces['secondsElapsed'] = .001 * (traces['timestamp'] - start_time)
                    traces.drop('timestamp', axis=1, inplace=True)
                    # get initial board info
                    board0 = ss.State().stringInitialize(row['stimuli'].values[0]['layout']) # create board object
                    value0 = board0.getHeuristic()
                    # update all actions 
                    boards = generate_boards(traces, "computed", board0 = board0)
                    traces['boxes'] = [b.crates for b in boards]
                    traces['state_key'] = [b.getKey2() for b in boards]
                    traces['state_value'] = [b.getHeuristic() for b in boards]
                    traces['restartCount'] = (traces['action'] == 'restart').cumsum()
                    # traces['value_diff'] = traces['state_value'].diff()
                    # traces.loc[traces.index[0], 'value_diff'] = traces.loc[traces.index[0], 'state_value'] - value0
                    # add first row for initial board
                    initial_row = pd.DataFrame([{'gameID': game, 'trialNum': trialnum, 'eventNum': None, 'action': None, 'event': None, 'boxes': board0.crates, 'agent.x': board0.player['x'], 'agent.y': board0.player['y'], 'agent.orientation': 'U', 'secondsElapsed': 0, 'state_key': board0.getKey2(), 'state_value': value0, 'restartCount': 0}])
                    df = pd.concat([initial_row, traces], ignore_index=True).reset_index(names='stateNum')
                    # merge and export to csv
                    traces = (S_tidy2[['gameID', 'condition','trialNum', 
                                       'stimuli_set', 'author_name', 'collection_name', 'level_name']]
                          .merge(df, how='right'))
                else:
                    traces = (S_tidy2[['gameID', 'condition','trialNum', 
                                       'stimuli_set', 'author_name', 'collection_name', 'level_name']]
                          .merge(traces, how='right'))
                # export to csv
                traces.to_csv(os.path.join(game_dir, game+"_test-"+ str(trialnum) + "_attemptTrace.csv"), index=False)

### Get Practice trial trace data

In [ ]:
# Trial info dataframe
if load_from_mongo:    
# ## fetch main solving records that match desire iterationNames, and turn it into a dataframe
    print("Fetching practice levels...")
    clear_output(wait=True)
    practice_cursor = coll.find({'study_metadata.project': project_name,
                'study_metadata.experiment': experiment_name,
                'study_metadata.iteration':  iterationName,#{'$in': iterationNames},
                'study_phase': 'practice'})
    practice = pd.DataFrame(list(practice_cursor))
    practice['trialNum'] = practice['trial_index'] - 3

    practice_tidy = (pd.DataFrame(practice.session_info.tolist())
                   .join(pd.DataFrame(practice[['trialNum','rt','startTime', 'time_elapsed', 'steps', 'inputEvents']], index=practice.index))
                   .join(pd.json_normalize(practice['stimuli']))
                   )
    
    practice_tidy = practice_tidy.assign(attempt_nsteps = practice_tidy['steps'].apply(lambda x: len(x) if isinstance(x, list) else pd.NA))
    practice_tidy['attempt_nInputEvents'] = practice_tidy['inputEvents'].apply(lambda x: len(x) if isinstance(x, list) else pd.NA)

    (practice_tidy.drop(['inputEvents'], axis=1)
     .to_csv(os.path.join(data_dir,iterationName+'_practiceTrials.csv'),index=False))
    
    print("Fetched {} practice trials from {} games".format(len(practice_tidy), len(practice_tidy.gameID.unique())))

# NOW EXPORT ATTEMPT TRACES
    print("Parsing puzzle-solving traces...")
    # clear_output(wait=True)

    # For each game, create a folder and save one csv per puzzle
    # in the format: gameID-xxx_trial-xxx_puzzle-xxx_inputevents.csv
    # where each row of the csv has input, game state

    for game in practice_tidy.gameID.unique():
        # make dir for each subject        
        game_dir = os.path.join(data_dir, "traces", game)
        make_dir_if_not_exists(game_dir)

        for trialnum in range(1, 4):
            row = practice_tidy[(practice_tidy['gameID'] == game) & (practice_tidy['trialNum'] == trialnum)]
            row['level_name'] = row['level_id'].astype(int, errors='ignore').astype(str)
            row['collection_name'] = row['collection_id']
            if not row.empty:
                print("parsing {} trial {}".format(game, trialnum))
                clear_output(wait=True)
                # build out traces object
                start_time = row['startTime'].item()
                traces = pd.json_normalize(row['inputEvents'].explode(ignore_index=True)).reset_index(names="eventNum")
                traces['gameID'] = game
                if 'timestamp' in traces.columns:
                    # update time column
                    traces['secondsElapsed'] = .001 * (traces['timestamp'] - start_time)
                    traces.drop('timestamp', axis=1, inplace=True)
                    # get initial board info
                    board0 = ss.State().stringInitialize(row['layout'].values[0]) # create board object
                    value0 = board0.getHeuristic()
                    # update all actions 
                    boards = generate_boards(traces, "computed", board0 = board0)
                    traces['boxes'] = [b.crates for b in boards]
                    traces['state_key'] = [b.getKey2() for b in boards]
                    traces['state_value'] = [b.getHeuristic() for b in boards]
                    traces['restartCount'] = (traces['action'] == 'restart').cumsum()
                    # traces['value_diff'] = traces['state_value'].diff()
                    # traces.loc[traces.index[0], 'value_diff'] = traces.loc[traces.index[0], 'state_value'] - value0
                    # add first row for initial board
                    initial_row = pd.DataFrame([{'gameID': game, 'eventNum': None, 'action': None, 'event': None, 'boxes': board0.crates, 'agent.x': board0.player['x'], 'agent.y': board0.player['y'], 'agent.orientation': 'U', 'secondsElapsed': 0, 'state_key': board0.getKey2(), 'state_value': value0, 'restartCount': 0}])
                    df = pd.concat([initial_row, traces], ignore_index=True).reset_index(names='stateNum')
                    # merge and export to csv
                    traces = (row[['gameID', 'condition','trialNum', 'collection_name', 'level_name']]
                          .merge(df, how='right'))
                else:
                    # empty row
                    traces = (row[['gameID', 'condition','trialNum', 'collection_name', 'level_name']]
                          .merge(traces, how='right'))
                # export to csv
                traces.to_csv(os.path.join(game_dir, game+"_practice-"+ str(trialnum) + "_attemptTrace.csv"), index=False)

traces.head()

### Get comparison trials

In [ ]:
if load_from_mongo:
    # # ## Now repeat for pre-test and post-test data
    print("Fetching comparison phase (pretest and posttest)...")
    clear_output(wait=True)
    c = coll.find({'study_metadata.project': project_name,
                'study_metadata.experiment': experiment_name,
                'study_metadata.iteration':  iterationName,#{'$in': iterationNames},
                'study_phase': {'$in': ['pretest', 'posttest']}})
    C = pd.DataFrame(list(c))


    C2 = (C[['study_phase', 'stimuli', 'trial_type', 'response', 'rt', 'rt_done1', 'rt_done2']]
            .join(pd.DataFrame(C.study_metadata.tolist(), index=C.index))
            .drop(['dev_mode', 'study_duration', 'compare_stim_duration'], axis=1)
            .join(pd.DataFrame(C.session_info.tolist(), index=C.index))
            .dropna(subset=['response']))

    C2 = C2.assign(trialNum = C['progress_prompt'].str.split(' ').str[1])
    C2["trialNum"] = C2["trialNum"].astype(int)
    C2["response"] = C2["response"].astype(int)
    # print(C2['response'].value_counts())


    stims = C2.explode('stimuli')['stimuli']
    stims = pd.json_normalize(stims).set_index(stims.index).reset_index(names="idx") # get original action index as a separate column
    stims['stimID'] = np.tile(['0','1'], int(len(stims)/2))

    # # pivot wider, so back to one event per row
    stims2 = stims.pivot(index='idx', columns = 'stimID', values = ['collection_name', 'level_name', 'layout'] )
    stims2.columns = stims2.columns.to_flat_index().str.join('_') # rename columns to x_1, x_2, etc.

    # # merge and export to csv   
    C_tidy = (C2[['gameID', 'condition', 'stim_id', 'study_phase', 'trial_type', 'trialNum', 'response', 'rt', 'rt_done1', 'rt_done2']]
                .join(stims2, how='inner')) # default to join by index

    ## only keep complete sessions
    if (only_completed):
        C_gameIDs = set([val['gameID'] for key, val in C['session_info'].items() if 'gameID' in val])
        if len(C_gameIDs) > len(completed_gameIDs):
            C_tidy = C_tidy[C_tidy['gameID'].isin(completed_gameIDs)]

    C_tidy.to_csv(os.path.join(data_dir,iterationName+'_compareTrials.csv'),index=False)

# check last output
C_tidy.head()

# Exclusions and replacements

In [ ]:
print("{} completed exit survey\n Now we look for missing data.".format(len(completed_gameIDs)))

## Find Missing Data

### Practice

In [ ]:
print("completed: {}".format(len(practice_tidy[practice_tidy['trialNum']==3])))
print("has all trace data: {}".format(len(practice_tidy.value_counts('gameID')[practice_tidy.value_counts('gameID') == 3])))
incomplete_practice = pd.DataFrame(practice_tidy.value_counts('gameID')[practice_tidy.value_counts('gameID') < 3])
print("incomplete/missing traces: {}".format(incomplete_practice))

# add to exclusions
incomplete_practice['reasons'] = incomplete_practice['count'].apply(lambda x: "Only {} practice trials".format(x))
exclusions = incomplete_practice.filter(['gameID', 'reasons']).reset_index()

### Comprehension

In [ ]:
# Passed comprehension
passed_comprehension = compre_tidy.query("correct==1")['gameID'].tolist()
print("passed: {}".format(len(passed_comprehension)))

print("When did participants fail the comprehension test?")
print(compre_tidy.groupby(['comprehension_attempt','correct'])['correct'].agg(['count']))

# Get gameIDs of participants who failed comprehension
failed_comprehension = compre_tidy.query("comprehension_attempt==2 & correct==0")['gameID'].tolist()
print("GameIDs that failed comprehension:", failed_comprehension)

failed_comprehension = pd.DataFrame(failed_comprehension, columns=['gameID']).assign(reasons="Failed comprehension")
print("New Exclusions: \n", failed_comprehension)
# add to exclusions
exclusions = pd.concat([exclusions, 
                        failed_comprehension], 
                       ignore_index=True)


### Test

In [ ]:
print("{} completed all 8 test trials".format(len(S_tidy2.groupby(['gameID'])['rate_response'].agg(['count']).query('count == 8'))))
# missing data
missing_test_ratings = S_tidy2.groupby(['gameID'])['rate_response'].agg(['count']).query('count < 8')
missing_test_ratings.reset_index(inplace=True)  
missing_test_ratings['reasons'] = "test ratings: only "+ missing_test_ratings['count'].astype(str) +" trials"
print("Missing test ratings: \n", missing_test_ratings)


missing_test_steps = S_tidy2.groupby(['gameID'])['attempt_nsteps'].agg(['count']).query('count < 8')
missing_test_steps.reset_index(inplace=True)  
missing_test_steps['reasons'] = "test attempts: only "+ missing_test_steps['count'].astype(str) +" trials"
print("Missing test steps: \n", missing_test_steps)

# # add to exclusions
exclusions = pd.concat([exclusions, 
                        missing_test_ratings[['gameID', 'reasons']],
                        missing_test_steps[['gameID', 'reasons']]],  
                       ignore_index=True)

### Comparison trials

In [ ]:
print("{} completed all 16 pre and post test trials".format(len(C_tidy.groupby(['gameID'])['response'].agg(['count']).query('count == 16'))))
# missing data
missing_comparisons = C_tidy.groupby(['study_phase', 'gameID'])['response'].agg(['count']).query('count < 8')

missing_comparisons.reset_index(inplace=True)  
missing_comparisons['reasons'] = "comparison: only "+missing_comparisons['count'].astype(str)+" "+missing_comparisons['study_phase']+" trials"

print("Missing comparison trials:", missing_comparisons)

# # add to exclusions
exclusions = pd.concat([exclusions, 
                        missing_comparisons[['gameID', 'reasons']]], 
                       ignore_index=True)

### Export exclusion list

In [ ]:
print("Exporting exclusion reasons")
exclusions.to_csv(os.path.join(data_dir,iterationName+'_exclusions.csv'),index=False)

## Identify Sessions with complete data

In [ ]:
final_gameIDs = set(completed_gameIDs) - set(exclusions['gameID'])
used_stimIDs = E_tidy['stim_id'][E_tidy['gameID'].isin(final_gameIDs)].reset_index(drop=True)

stim_cursor = conn['stimuli'][experiment_name]
stim_collection = pd.DataFrame(list(stim_cursor.find()))
stim_collection['stim_id'] = stim_collection.apply(lambda row: str(row['_id']), axis=1)
stim_collection_used = stim_collection[stim_collection['stim_id'].isin(used_stimIDs)]

# Counterbalancing so far
print("{} valid sessions using {} unique stim sequences, after {} requests to Mongo stimuli db".format(
    len(final_gameIDs), len(stim_collection_used), np.sum(len(stim_collection['games']))
    ))

In [ ]:
counterbalancing = E_tidy[E_tidy['gameID'].isin(final_gameIDs)].merge(stim_collection_used[['stim_id', 'stimuli_set_order']])
print("Counterbalancing table of valid sessions:")
print(counterbalancing.pivot_table(index='stimuli_set_order', columns='condition', values='gameID', aggfunc='count'))

print("Unique stimuli requested and used")
print(stim_collection_used.pivot_table(index='stimuli_set_order', columns='condition', values='numGames', aggfunc='count'))

In [ ]:
print("Which stimuli were used repeatedly?")
repeated_stims = counterbalancing.value_counts('stim_id')[counterbalancing.value_counts('stim_id') > 1].reset_index()
print(repeated_stims)

print("Which stimuli were not used?")
stim_collection_unused = stim_collection[~stim_collection['stim_id'].isin(used_stimIDs)].reset_index()
print(stim_collection_unused.pivot_table(index='stimuli_set_order', columns='condition', values='numGames', aggfunc='count'))

### Do replacements

In [ ]:
if do_replacements:
  print("Setting used stimuli to numGames = 100")
  from bson.objectid import ObjectId
  for stringid in counterbalancing['stim_id']:
  # for stringid in stim_collection_unused['stim_id']:
  # for stringid in pd.concat([counterbalancing['stim_id'],repeated_stims['stim_id']],ignore_index=True).unique():
    stim_cursor.update_one({
      '_id': ObjectId(stringid)
    },{
      '$set': {
        'numGames': 100
        # 'numGames': 0
      }
    }, upsert=False)


  

In [ ]:
print("Now in MongoDB Stims:")
print("{} stim have numGames =100".format(len(list(stim_cursor.find({'numGames': {'$eq': 100} })))))
print("{} stim have numGames < 10".format(len(list(stim_cursor.find({'numGames': {'$lt': 10} })))))  # Check remaining stim in MongoDB:") 
print("{} stim have numGames <2".format(len(list(stim_cursor.find({'numGames': {'$lt': 2} })))))  # Check remaining stim in MongoDB:") 

# pd.DataFrame(list(stim_cursor.find({'numGames': {'$lt': 10} }))).pivot_table(index='stimuli_set_order', columns='condition', values='_id', aggfunc='count')


# <a name="read_from_csv"></a> Read Dataframes Back Up ([^](#top))

In [ ]:
survey = pd.read_csv(os.path.join(data_dir,iterationName+'_survey.csv'))
test = pd.read_csv(os.path.join(data_dir,iterationName+'_testTrials.csv'))
compare = pd.read_csv(os.path.join(data_dir,iterationName+'_compareTrials.csv'))
comprehension = pd.read_csv(os.path.join(data_dir,iterationName+'_compareTrials.csv'))
exclusions = pd.read_csv(os.path.join(data_dir,iterationName+'_exclusions.csv'))

print("** {} participants completed the exit survey.".format(survey.gameID.nunique()))
print(survey['condition'].value_counts())
print(survey['participantGender'].value_counts())
print("M_age = {}, SD_age = {}".format(
    np.round(survey['participantYears'].mean(),2), 
    np.round(survey['participantYears'].std(),2)))

In [ ]:
survey_included = survey[~survey['gameID'].isin(exclusions['gameID'])]
print("** {} participants have usable data.".format(survey_included.gameID.nunique()))
print(survey_included['condition'].value_counts())
print(survey_included['participantGender'].value_counts())
print("M_age = {}, SD_age = {}".format(
    np.round(survey_included['participantYears'].mean(),2), 
    np.round(survey_included['participantYears'].std(),2)))

### check for complaints

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(survey[['condition','technicalDifficultiesComments']].dropna().sort_values(by=['technicalDifficultiesComments']))

## <a name="summary"></a> Summary of the Data ([^](#top))

In [ ]:
print(survey_included.groupby(['experiment', 'iteration', 'condition']).size().reset_index(name='count'))

print("N Test Trials: ")
print(test[~test['gameID'].isin(exclusions['gameID'])].groupby(['condition', 'gameID'])['solved'].agg(['count']).value_counts('count'))

# Which stimuli called more than once?
survey_included['stim_id'].value_counts()[survey_included['stim_id'].value_counts() > 1]

In [ ]:
# mean solved should be similar across conditions
print('Time to solution by condition')
print(test[~test['gameID'].isin(exclusions['gameID'])].groupby(['condition'])['solveDuration'].agg(['count', 'mean', 'std']))
# print('Solution duration by puzzle')
# print(test.groupby(['collection_name','level_name', 'condition'])['solveDuration'].agg(['count', 'mean', 'std']))

In [ ]:
print('Solution rate by puzzle')
print(test[~test['gameID'].isin(exclusions['gameID'])]
      .groupby(['collection_name','level_name'])['solveDuration']
      .agg(['count', 'mean', 'std'])
      .sort_values(by = 'count'))
# print(solve.loc[solve['trial_type']=='sokoban-solve'].groupby(['stim_collectionName', 'stim_levelId'])['solveDuration'].agg(['count', 'mean', 'std']))
